In [1]:
# Notebook makes finds distribution of datasets after undersampling 1:3, before train/test split

In [2]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

In [3]:
from sklearn.linear_model import LogisticRegression
from io import StringIO
import sys
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold

In [4]:
all_assay_all_compounds =pd.read_csv("Predictions_train_heldout_scaled_prob_all_assays.csv")
all_assay_all_compounds

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.853763,1,0.481794,0.893879,1,0.269814,1.0,Training,True,True,0.858900,0.927333
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.357098,0,0.481794,0.412432,1,0.269814,1.0,Training,False,True,0.370592,0.597659
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.046336,0,0.481794,0.167905,0,0.269814,0.0,Training,True,True,0.048086,0.311150
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.039431,0,0.481794,0.182939,0,0.269814,0.0,Training,True,True,0.040921,0.339009
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.345300,0,0.481794,0.091189,0,0.269814,0.0,Training,True,True,0.358348,0.168984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,InChI=1S/C12H11N3O2S/c1-8-7-9(2)15(13-8)12-10-...,0.127744,0,0.245120,0.513325,1,0.428485,0.0,HeldOut,True,False,0.260575,0.574224
58483,93_259,InChI=1S/C17H16N2O4S/c1-22-14-7-8-16(23-2)17(1...,0.505973,1,0.245120,0.322499,0,0.428485,0.0,HeldOut,False,True,0.672778,0.376325
58484,93_259,InChI=1S/C24H21N3O2S/c28-23-19-8-4-5-9-21(19)3...,0.363862,1,0.245120,0.359675,0,0.428485,1.0,HeldOut,True,False,0.578650,0.419705
58485,93_259,InChI=1S/C18H22N2O4S/c1-14(2)15-5-7-16(8-6-15)...,0.290175,1,0.245120,0.260526,0,0.428485,0.0,HeldOut,False,True,0.529843,0.304008


In [5]:
len(all_assay_all_compounds.assay.unique())

89

In [6]:
stats_list=[]
for assay in all_assay_all_compounds.assay.unique():   
    per_assay_compounds=all_assay_all_compounds[all_assay_all_compounds["assay"]==assay]
    inactives= per_assay_compounds["true"].value_counts()[0]
    actives= per_assay_compounds["true"].value_counts()[1]
    total=inactives+actives
    
    row=[assay, total, actives, inactives]
    stats_list.append(row)

In [7]:
df = pd.DataFrame(stats_list, columns=["assay", "total", "actives", "inactives"])
df["ratio"] = df["actives"]/df["total"]
df

,assay,total,actives,inactives,ratio
0,178_684,272,68,204,0.250000
1,180_686,204,51,153,0.250000
2,183_604,180,45,135,0.250000
3,184_606,312,78,234,0.250000
4,185_665,120,30,90,0.250000
...,...,...,...,...,...
84,278_757,1508,377,1131,0.250000
85,279_758,264,66,198,0.250000
86,280_759,540,135,405,0.250000
87,283_762,428,107,321,0.250000


In [8]:
df.describe()

,total,actives,inactives,ratio
count,89.000000,89.000000,89.000000,89.000000
mean,657.157303,164.314607,492.842697,0.250199
std,671.306002,167.806476,503.499749,0.001878
min,104.000000,26.000000,78.000000,0.250000
25%,180.000000,45.000000,135.000000,0.250000
50%,384.000000,96.000000,288.000000,0.250000
75%,832.000000,208.000000,624.000000,0.250000
max,3064.000000,766.000000,2298.000000,0.267717


In [9]:
df.to_csv("assay_compounds_stats_list_bothsideundersampling.csv", index=False)